# Chapter 5 pandas 시작하기

- pandas는 아래의 도구들과 같이 사용하는 경우가 흔하다.
    - 산술 계산 도구인 NumPy와 SciPy
    - 분석 라이브러리인 statsmodels와 scikit-learn
    - 시각화 도구인 matplotlib  
- pandas는 for 문을 사용하지 않고 데이터를 처리한다거나 배열 기반의 함수를 제공하는 등 NumPy의 배열 기반 계산 스타일을 많이 차용했다.
- pandas의 import 컨벤션은 `import pandas as pd`
- E또한 `Series`와 `DataFrame`은 로컬 네임스페이스로 임포트하는 것이 컨벤션

In [1]:
import pandas as pd
from pandas import Series, DataFrame

## 5.1 pandas 자료구조 소개

- pandas에 대해 알아보려면 Series와 DataFrame, 이 두 가지 자료구조에 익숙해질 필요가 있다.

### 5.1.1 Series

- Series는 일련의 객체를 담을 수 있는 1차원 배열 같은 자료구조다 (어떤 NumPy 자료형이라도 담을 수 있다).
- $색인^{\text{index}}$이라고 하는 배열의 데이터와 연관된 `이름`을 가지고 있다.
- 가장 간단한 Series 객체는 배열 데이터로부터 생성할 수 있다.

In [2]:
obj = pd.Series([4,7,-5,3])
obj

0    4
1    7
2   -5
3    3
dtype: int64

In [3]:
obj.values

array([ 4,  7, -5,  3], dtype=int64)

In [4]:
obj.index  # range(4)와 같다.

RangeIndex(start=0, stop=4, step=1)

> 별도로 index를 지정해 줄 수 있다.

In [5]:
obj2 = pd.Series([4,7,-5,3], index = ['d','b','a','c'])
obj2

d    4
b    7
a   -5
c    3
dtype: int64

In [6]:
obj2.index

Index(['d', 'b', 'a', 'c'], dtype='object')

> 다음과 같은 활용 방안도 있다.

In [7]:
import numpy as np

print('''
obj2['a'] : {0} \n

------------------

obj2[['c','a','d']] 

{1} 

------------------

obj2[obj2 > 0] 

{2}
    
------------------
    
np.exp(obj2) 

{3}'''.format(obj2['a'], \
              obj2[['c','a','d']], \
              obj2[obj2 > 0], \
              np.exp(obj2)))


obj2['a'] : -5 


------------------

obj2[['c','a','d']] 

c    3
a   -5
d    4
dtype: int64 

------------------

obj2[obj2 > 0] 

d    4
b    7
c    3
dtype: int64
    
------------------
    
np.exp(obj2) 

d      54.598150
b    1096.633158
a       0.006738
c      20.085537
dtype: float64


> Series는 색인값에 데이터값을 매핑하고 있으므로 파이썬의 사전형과 비슷하다.  
> Series 객체는 파이썬의 사전형을 인자로 받아야 하는 많은 함수에서 사전형을 대체하여 사용할 수 있다.

In [8]:
sdata = {'Ohio' : 35000, 'Texas' : 71000, 'Oregon' : 16000, 'Utah' : 5000}

obj3 = pd.Series(sdata)
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

> 색인을 직접 지정하고 싶다면 원하는 순서대로 색인을 직접 넘겨줄 수도 있다.

In [9]:
states = ['California','Ohio','Oregon','Texas']

obj4 = pd.Series(sdata, index = states)
obj4 

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

> 위 결과에서 California index에 해당하는 값이 없으므로 $\text{NAN}^{\text{not a number}}$으로 표시된다.  
> 'Utah'는 states에 포함되어 있지 않으므로 실행 결과에서 빠지게 된다.

> pandas의 isnull과 notnull 함수는 누락된 데이터를 찾을 때 사용된다.

In [10]:
pd.isnull(obj4)

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [11]:
pd.notnull(obj4)

California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool

In [12]:
# 해당 메서드는 Series 객체의 인스턴트 메서드로도 존재한다.
obj4.isnull()

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

> Series의 유용한 기능은 산술 연산에서 색인과 라벨로 자동 정렬하는 것이다.  
> 데이터베이스를 사용해본 경험이 있다면 join 연산과 비슷하다고 여겨질 것이다.

In [13]:
obj3 + obj4

California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

> Series 객체와 Series의 색인은 모두 name 속성이 있음. 이 속성은 pandas의 핵심 기능과 밀접한 관련이 있다.

In [14]:
obj4.name = 'population'
obj4.index.name = 'state'

obj4

state
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: population, dtype: float64

> Series의 색인은 대입하여 변경할 수 있다.

In [15]:
obj.index = ['Bob','Steve','Jeff','Ryan']
obj

Bob      4
Steve    7
Jeff    -5
Ryan     3
dtype: int64

### 5.1.2 DataFrame
- DataFrame은 표 같은 스프레드시트 형식의 자료구조
- 여러 개의 컬럼이 있으며 각 컬럼은 서로 다른 종류의 값 (숫자, 문자열, 불리언 등)을 담을 수 있음
- DataFrame은 로우와 컬럼에 대한 색인을 가지고 있음. 색인의 모양이 같은 Series 객체를 담고 있는 파이썬 사전으로 생각하면 편하다.
- 물리적으로 DataFrame은 2차원이지만 계층적 색인을 이용해서 좀 더 고차원의 데이터를 표현할 수 있음
- DataFrame 객체를 생성하는 가장 흔한 방법은 같은 길이의 리스트에 담긴 사전을 이용하거나 NumPy 배열을 이용하는 것이다.

In [16]:
data = {'state' : ['Ohio','Ohio','Ohio','Nevada','Nevada','Nevada'],
        'year' : [2000,2001,2002,2001,2002,2003],
        'pop' : [1.5,1.7,3.6,2.4,2.9,3.2]}
frame = pd.DataFrame(data)
frame

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


> 원하는 순서대로 columns를 지정하면 원하는 순서를 가진 DataFrame 객체가 `생성`된다.

In [17]:
pd.DataFrame(data, columns = ['year','state','pop'])

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9
5,2003,Nevada,3.2


> Series와 마찬가지로 사전에 없는 값을 넘기면 결측치로 저장된다.

In [18]:
frame2 = pd.DataFrame(data, 
                      columns = ['year','state','pop','debt'],
                      index = ['one','two','three','four','five','six'])
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,NaN
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,NaN
five,2002,Nevada,2.9,NaN
six,2003,Nevada,3.2,NaN


In [19]:
frame2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

> DataFrame의 컬럼은 Series처럼 `사전 형식의 표기법`으로 접근하거나 `속성 형식`으로 접근할 수 있다.

In [20]:
frame2['state']   # 사전 형식의 표기법으로 컬럼에 접근하는 방법

one        Ohio
two        Ohio
three      Ohio
four     Nevada
five     Nevada
six      Nevada
Name: state, dtype: object

In [21]:
frame2.year # 속성 형식으로 컬럼에 접근하는 방법

one      2000
two      2001
three    2002
four     2001
five     2002
six      2003
Name: year, dtype: int64

> 로우는 위치나 loc 속성을 이용해서 이름을 통해 접근할 수 있다.

In [22]:
frame2.loc['three']

year     2002
state    Ohio
pop       3.6
debt      NaN
Name: three, dtype: object

> 컬럼은 스칼라값이나 배열의 값으로 대입 가능하다.  
> 리스트나 배열을 컬럼에 대입할 때는 대입하려는 값의 길이가 DataFrame의 크기와 `동일`해야 한다.

In [23]:
frame2['debt'] = 16.5
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,16.5
two,2001,Ohio,1.7,16.5
three,2002,Ohio,3.6,16.5
four,2001,Nevada,2.4,16.5
five,2002,Nevada,2.9,16.5
six,2003,Nevada,3.2,16.5


In [24]:
frame2['debt'] = np.arange(6.)
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,0.0
two,2001,Ohio,1.7,1.0
three,2002,Ohio,3.6,2.0
four,2001,Nevada,2.4,3.0
five,2002,Nevada,2.9,4.0
six,2003,Nevada,3.2,5.0


> Series를 컬럼에 대입하는 경우 DataFrame의 색인에 따라 값이 대입되며 존재하지 않는 색인에는 결측치가 대입된다.

In [25]:
val = pd.Series([-1.2, -1.5, -1.7], index = ['two','four','five'])

frame2['debt'] = val
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,-1.2
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,-1.5
five,2002,Nevada,2.9,-1.7
six,2003,Nevada,3.2,NaN


> 존재하지 않는 컬럼을 대입하면 새로운 컬럼을 생성한다.  
> del 예약어를 사용해서 컬럼을 삭제할 수 있다.

In [26]:
frame2['eastern'] = frame2['state'] == 'Ohio'
frame2

,year,state,pop,debt,eastern
one,2000,Ohio,1.5,NaN,True
two,2001,Ohio,1.7,-1.2,True
three,2002,Ohio,3.6,NaN,True
four,2001,Nevada,2.4,-1.5,False
five,2002,Nevada,2.9,-1.7,False
six,2003,Nevada,3.2,NaN,False


In [27]:
del frame2['eastern']
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,-1.2
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,-1.5
five,2002,Nevada,2.9,-1.7
six,2003,Nevada,3.2,NaN


> 중첩된 사전을 이용해서 데이터를 생성할 수 있다.  
> 중첩된 사전의 `바깥에 있는 키`는 `컬럼`이 되고 `안에 있는 키`는 `로우(인덱스)`가 된다.

In [28]:
pop = {'Nevada' : {2001 : 2.4, 2002 : 2.9},
       'Ohio' : {2000 : 1.5, 2001 : 1.7, 2002 : 3.6}}

frame3 = pd.DataFrame(pop)
frame3

,Nevada,Ohio
2001,2.4,1.7
2002,2.9,3.6
2000,NaN,1.5


> 만일 데이터 프레임의 색인(index)과 컬럼(columns)에 name 속성을 지정했다면 이 역시 함께 출력됨

In [29]:
frame3.index.name = 'year'; frame3.columns.name = 'state'
frame3

state,Nevada,Ohio
year,,
2001,2.4,1.7
2002,2.9,3.6
2000,NaN,1.5


> Series와 유사하게 values 속성은 DataFrame에 저장된 데이터를 2차원 배열로 반환함

In [30]:
frame3.values

array([[2.4, 1.7],
       [2.9, 3.6],
       [nan, 1.5]])

#### DataFrame 생성자에 넘길 수 있는 자료형 목록

<details>
<summary>DataFrame 생성을 위한 입력 데이터의 종류</summary>
<div markdown="1">

|형|설명|
|:--|:--|
|2차원 ndarray|데이터를 담고 있는 행렬. 선택적으로 행(로우)과 열(컬럼)의 이름을 전달할 수 있다.|
|배열, 리스트, 튜플의 사전|사전의 모든 항목은 같은 길이를 가져야 하며, 각 항목의 내용이 DataFrame의 컬럼이 된다.|
|NumPy의 구조화 배열|배열의 사전과 같은 방식으로 취급된다.|
|Series의 사전|Series의 각 값이 컬럼이 된다. 명시적으로 색인을 넘겨주지 않으면 각 Series의 색인이 하나로 합쳐져서 로우의 색인이 된다.|
|사전의 사전|내부에 있는 사전이 컬럼이 된다. 키값은 'Series의 사전'과 마찬가지로 합쳐져서 로우의 색인이 된다.|
|사전이나 Series의 리스트|리스트의 각 항목이 DataFrame의 로우가 된다. 합쳐진 사전의 키값이나 Series의 색인이 DAtaFrame의 컬럼 이름이 된다.|
|리스트나 튜플의 리스트|'2차원 ndarray'의 경우와 같은 방식으로 취급된다.|
|다른 DataFrame|색인을 따로 지정하지 않으면 DataFrame의 색인이 그대로 사용된다.|
|NumPy MaskedArray|'2차원 ndarray'의 경우와 같은 방식으로 취급되지만 마스크값은 반환되는 DataFrame에서 NA값이 된다.|
    
</div>
</details>

### 5.1.3 색인 객체 
- pandas의 색인 객체는 표 형식의 데이터에서 `각 로우와 컬럼에 대한 이름`과 다른 메타데이터 (`축의 이름 등`)을 저장하는 `객체`다.
- Series나 DataFrame 객체를 생성할 때 사용되는 배열이나 다른 순차적인 이름은 내부적으로 색인으로 변환된다.

In [31]:
obj = pd.Series(range(3), index = ['a','b','c'])
index = obj.index

In [32]:
print('''
index : {0}
index[1:] : {1}'''.format(index, index[1:]))


index : Index(['a', 'b', 'c'], dtype='object')
index[1:] : Index(['b', 'c'], dtype='object')


> 색인 객체는 변경이 불가능하다.

In [33]:
index[1] = 'd'

TypeError: Index does not support mutable operations

> 그러므로 자료구조 사이에서 안전하게 공유될 수 있다.

In [34]:
labels = pd.Index(np.arange(3))
labels

Int64Index([0, 1, 2], dtype='int64')

In [35]:
obj2 = pd.Series([1.5,-2.5,0], index = labels)
obj2

0    1.5
1   -2.5
2    0.0
dtype: float64

In [36]:
obj2.index is labels

True

#### 색인 메서드와 속성

<details>
<summary>색인 메서드와 속성</summary>
<div markdown="1">

|메서드|설명|
|:--|:--|
|append|추가적인 색인 객체를 덧붙여 새로운 색인을 반환한다.|
|difference|색인의 차집합을 반환한다.|
|intersection|색인의 교집합을 반환한다.|
|union|색인의 합집합을 반환한다.|
|isin|색인이 넘겨받은 색인에 존재하는지 알려주는 불리언 배열을 반환한다.|
|delete|i 위치의 색인이 삭제된 새로운 색인을 반환한다.|
|drop|넘겨받은 값이 삭제된 새로운 색인을 반환한다.|
|insert|i 위치에 색인이 추가된 새로운 색인을 반환한다.|
|is_monotonic|색인이 단조성을 가진다면 True를 반환한다.|
|is_unique|중복되는 색인이 없다면 True를 반환한다.|
|unique|색인에서 중복되는 요소를 제거하고 유일한 값만 반환한다.|
    
</div>
</details>

## 5.2 핵심 기능

### 5.2.1 재색인
- reindex : 새로운 색인에 맞도록 객체를 `새로 생성`한다.

In [37]:
obj = pd.Series([4.5,7.2,-5.3,3.6], index = ['d','b','a','c'])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

> Series 객체에 대해 reindex를 호출하면 데이터를 새로운 색인에 맞게 재배열함.  
> 존재하지 않는 색인값이 있다면 NaN을 추가함.

In [38]:
obj2 = obj.reindex(['a','b','c','d','e'])
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

> 시계열 같은 순차적인 데이터를 재색인할 때 값을 보간하거나 채워 넣어야 할 경우가 있다.  
> 이는 method 옵션을 이용해서 해결할 수 있다.  
> ffill 메서드는 누락된 값을 직전의 값으로 채워 넣는 방법을 의미한다.

In [39]:
obj3 = pd.Series(['blue','purple','yellow'], index = [0,2,4])
obj3

0      blue
2    purple
4    yellow
dtype: object

In [40]:
obj3.reindex(range(6), method = "ffill")

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

>  DataFrame에 대한 reindex는 로우(색인), 컬럼 또는 둘 다 변경이 가능함.  
> 그냥 순서만 전달하면 로우가 재색인된다.

In [41]:
frame = pd.DataFrame(np.arange(9).reshape((3,3)),
                     index = ['a','c','d'],
                     columns = ['Ohio','Texas','California'])
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [42]:
frame2 = frame.reindex(['a','b','c','d'])
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


> 컬럼은 columns 예약어를 사용해서 재색인할 수 있다.

In [43]:
states = ['Texas', 'Utah', 'California']

frame.reindex(columns = states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


#### reindex의 인자

<details>
<summary>재색인 함수 인자</summary>
<div markdown="1">

|인자|설명|
|:--|:--|
|index|색인으로 사용할 새로운 순서. Index 인스턴스나 다른 순차적인 자료구조가 사용 가능하다. Index는 복사가 이루어지지 않고 그대로 사용된다.|
|method|채움 메서드. ffill은 직전 값을 채워 넣고 bfill은 다음 값을 채워 넣는다.|
|fill_value|재색인 과정 중에 새롭게 나타나는 비어 있는 데이터를 채우기 위한 값|
|limit|전/후 보간 시에 사용할 최대 갭 크기 (채워넣을 원소의 수)|
|tolerance|전/후 보간 시에 사용할 최대 갭 크기 (값의 차이)|
|level|MultiIndex의 단계(level)에 단순 색인을 맞춘다. 그렇지 않으면 MultiIndex의 하위집합에 맞춘다.|
|copy|True인 경우 새로운 색인이 이전 색인과 동일하더라도 데이터를 복사한다. False인 경우 새로운 색인이 이전 색인과 동일할 경우 복사하지 않는다.|   
    
</div>
</details>

### 5.2.2. 하나의 로우나 컬럼 삭제하기

- drop 메서드를 사용하면 선택한 값들이 삭제된 `새로운 객체` 를 얻을 수 있다.

In [44]:
obj = pd.Series(np.arange(5.), index = ['a','b','c','d','e'])

obj.drop(['d','c'])

a    0.0
b    1.0
e    4.0
dtype: float64

> DataFrame에서는 로우와 컬럼 모두에서 값을 삭제할 수 있다.

In [45]:
data = pd.DataFrame(np.arange(16).reshape((4,4)),
                    index = ['Ohio','Colorado','Utah','New York'],
                    columns = ['one','two','three','four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


> drop 함수 인자에 로우 이름을 넘기면 해당 로우 (axis 0)의 값을 모두 삭제한다.

In [46]:
data.drop(['Colorado','Ohio'])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


> 컬럼의 값을 삭제할 때는 axis = 1 또는 axis = 'columns'를 인자로 넘겨주면 된다.

In [47]:
data.drop('two', axis = 1)

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


In [48]:
data.drop(['two','four'], axis = 'columns')

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


> inplace 옵션을 사용하면 새로운 객체를 반환하는 대신 원본 객체를 변경한다.

In [49]:
data.drop(['two','four'], axis = 'columns', inplace = True)

In [50]:
data

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


### 5.2.3 색인하기, 선택하기, 거르기

> Series의 색인은 NumPy 배열의 색인과 유사하게 동작하지만 정수가 아니어도 된다는 점이 다르다.

In [51]:
obj = pd.Series(np.arange(4.), index = ['a','b','c','d'])

In [52]:
obj[2:4]

c    2.0
d    3.0
dtype: float64

In [53]:
obj[['b','c']]

b    1.0
c    2.0
dtype: float64

In [54]:
obj[[1,3]]

b    1.0
d    3.0
dtype: float64

In [55]:
obj[obj<2]

a    0.0
b    1.0
dtype: float64

In [56]:
obj['b':'c'] # 라벨 이름으로 슬라이싱하면 시작점과 끝점을 포함한다는 것이 일반 파이썬에서의 슬라이싱과 다른 점 

b    1.0
c    2.0
dtype: float64

> 색인으로 DataFrame에서 하나 이상의 컬럼 값을 가져올 수 있다.

In [57]:
data = pd.DataFrame(np.arange(16).reshape((4,4)), 
                    index = ['Ohio','Colorado','Utah','New York'],
                    columns = ['one','two','three','four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [58]:
data[['three','one']]

,three,one
Ohio,2,0
Colorado,6,4
Utah,10,8
New York,14,12


> 슬라이싱으로 로우를 선택하거나 불리언 배열로 로우를 선택할 수 있다.

In [59]:
data[:2]

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


In [60]:
data[data['three'] > 5]

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


#### loc과 iloc으로 선택하기
- loc과 iloc는 DataFrame의 `로우`에 대해 `라벨`로 색인할 수 있게 해주는 특수한 색인 필드
- 이 방법을 이용하면 NumPy와 비슷한 방식으로 추가적으로 `축의 라벨`을 사용하여 DataFrame의 로우와 컬럼을 선택할 수 있다.
- `축 이름`을 선택할 때는 `loc`을, `정수 색인`으로 선택할 때는 `iloc`을 사용한다.

In [61]:
data.loc[['Colorado','Utah'], ['two','three']]

,two,three
Colorado,5,6
Utah,9,10


In [62]:
data.iloc[[1,2], [3,0,1]]

,four,one,two
Colorado,7,4,5
Utah,11,8,9


> 이 두 함수는 슬라이스도 지원할 뿐더러 단일 라벵리나 라벨 리스트도 지원한다.

In [63]:
data.loc[:"Utah",:"two"]

,one,two
Ohio,0,1
Colorado,4,5
Utah,8,9


In [64]:
data.iloc[:,:3][data.three > 5]

,one,two,three
Colorado,4,5,6
Utah,8,9,10
New York,12,13,14


#### pandas 객체에서 데이터를 선택하고 재배열하는 방법

<details>
<summary>DataFrame의 값 선택하기</summary>
<div markdown="1">

|방식|설명|
|:--|:--|
|df[val]|DataFrame에서 하나의 컬럼 또는 여러 컬럼을 선택한다. 편의를 위해 불리언 배열, 슬라이스, 불리언 DataFrame을 사용할 수 있다.|
|df.loc[val]|DataFrame에서 라벨값으로 `로우`의 부분집합을 선택한다.|
|df.loc[:,val]|DataFrame에서 라벨값으로 `컬럼`의 부분집합을 선택한다.|
|df.loc[val1,val2]|DataFrame에서 라벨값으로 `로우와 컬럼`의 부분집합을 선택한다.|
|df.iloc[where]|DataFrame에서 정수 색인으로 `로우`의 부분집합을 선택한다.|
|df.iloc[:, where]|DataFrame에서 정수 색인으로 `컬럼`의 부분집합을 선택한다.|
|df.iloc[where_i,where_j]|DataFrame에서 정수 색인으로 `로우와 컬럼`의 부분집합을 선택한다.|
|df.at[label_i, label_j]|로우와 컬럼의 라벨로 `단일 값`을 선택한다.|
|df.iat[i,j]|로우와 컬럼의 정수 색인으로 `단일 값`을 선택한다.|
|reindex 메서드|하나 이상의 축을 새로운 색인으로 맞춘다.|
|get_value, set_value 메서드|로우와 컬럼 이름으로 DataFrame의 값을 선택한다.|
    
</div>
</details>

### 5.2.4 정수 색인

- pandas 객체는 일관성을 유지하기 위해 정숫값을 담고 있는 축 색인이 있다면 우선적으로 라벨을 먼저 찾아 보도록 구현되어 있다.
- 좀 더 세밀하게 사용하고 싶다면 라벨에 대해서는 loc을 사용하고 정수 색인에 대해서는 iloc을 사용하자.

In [65]:
ser = pd.Series(np.arange(3.))

In [66]:
ser[:1]

0    0.0
dtype: float64

In [67]:
ser.loc[:1]

0    0.0
1    1.0
dtype: float64

In [68]:
ser.iloc[:1]

0    0.0
dtype: float64

### 5.2.5 산술 연산과 데이터 정렬

- pandas에서 가장 중요한 기능 중 하나는 다른 색인을 가지고 있는 객체 간의 산술 연산
- 객체를 더할 때 `짝이 맞지 않는 색인`이 있따면 결과에 `두 색인이 통합`된다.
- 데이터베이스의 외부 조인과 유사하게 동작한다고 생각할 수 있다.

> 서로 겹치는 색인이 없는 경우 데이터는 NA 값이 된다.  
> 산술 연산 시 누락된 값은 전파된다.

In [69]:
s1 = pd.Series([7.3,-2.5,3.4,1.5], index = ['a','c','d','e'])
s2 = pd.Series([-2.1,3.6,-1.5,4,3.1],
               index = ['a','c','e','f','g'])

s1 + s2

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

> DataFrame을 더하면 DataFrame에 있는 색인과 컬럼이 하나로 합쳐진다.  
> 양쪽 DataFrame 객체에 모두 존재하지 않는 라벨에 대해서는 없는 값으로 나타난다. 즉, 공통되는 컬럼이나 로우가 아닌 경우 결과에 아무 것도 나타나지 않는다.

In [70]:
df1 = pd.DataFrame(np.arange(9.).reshape((3,3)), columns = list('bcd'),
                   index = ['Ohio','Texas','Colorado'])

df2 = pd.DataFrame(np.arange(12.).reshape((4,3)), columns = list('bde'),
                   index = ['Utah','Ohio','Texas','Oregon'])

In [71]:
df1

,b,c,d
Ohio,0.0,1.0,2.0
Texas,3.0,4.0,5.0
Colorado,6.0,7.0,8.0


In [72]:
df2

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [73]:
df1 + df2

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,NaN,6.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,NaN,12.0,NaN
Utah,NaN,NaN,NaN,NaN


#### 산술 연산 메서드에 채워 넣을 값 지정하기
- 서로 다른 색인을 가지는 객체 간의 산술 연산에서 존재하지 않는 축의 값을 특수한 값으로 지정하고 싶을 때는 다음과 같이 할 수 있다.

In [74]:
df1 = pd.DataFrame(np.arange(12.).reshape((3,4)),
                   columns = list('abcd'))

df2 = pd.DataFrame(np.arange(20.).reshape((4,5)), 
                   columns = list('abcde'))
df2.loc[1,'b'] = np.nan

In [75]:
df1

,a,b,c,d
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0


In [76]:
df2

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,4.0
1,5.0,NaN,7.0,8.0,9.0
2,10.0,11.0,12.0,13.0,14.0
3,15.0,16.0,17.0,18.0,19.0


In [77]:
df1 + df2   # 이 둘을 더하면 겹치지 않는 부분은 NA값이 된다.

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,NaN
1,9.0,NaN,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


> df1에 add 메서드를 사용하고, df2와 fill_value 값을 인자로 전달한다.

In [78]:
df1.add(df2, fill_value = 0)

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,5.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


> Series나 DataFrame을 재색인할 때도 fill_value를 지정할 수 있다.

In [79]:
df1.reindex(columns = df2.columns)

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,NaN
1,4.0,5.0,6.0,7.0,NaN
2,8.0,9.0,10.0,11.0,NaN


In [80]:
df1.reindex(columns = df2.columns, fill_value = 0 )

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,0
1,4.0,5.0,6.0,7.0,0
2,8.0,9.0,10.0,11.0,0


#### 산술 연산 메서드 

<details>
<summary>산술 연산 메서드</summary>
<div markdown="1">

|메서드|설명|
|:--|:--|
|add, radd| 덧셈(+)을 위한 메서드|
|sub, rsub| 뺄셈(-)을 위한 메서드|
|div, rdiv| 나눗셈(/)을 위한 메서드|
|floordiv, rfloordiv| 소숫점 내림(//) 연산을 위한 메서드|
|mul, rmul| 곱셈(*)을 위한 메서드|
|pow, rpow| 멱승(**)을 위한 메서드|
    
</div>
</details>

#### DataFrame과 Series 간의 연산

> 다른 차원의 NumPy 배열과 연산처럼 DataFrame과 Series 간의 연산도 잘 정의되어 있다.  
> 기본적으로 DataFrame과 Series 간의 산술 연산은 Series의 색인을 DataFrame의 컬럼에 맞추고 아래 로우로 전파한다.

In [81]:
frame = pd.DataFrame(np.arange(12.).reshape((4,3)),
                     columns = list('bde'),
                     index = ['Utah','Ohio','Texas','Oregon'])
series = frame.iloc[0]

In [82]:
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [83]:
series

b    0.0
d    1.0
e    2.0
Name: Utah, dtype: float64

In [84]:
frame - series

,b,d,e
Utah,0.0,0.0,0.0
Ohio,3.0,3.0,3.0
Texas,6.0,6.0,6.0
Oregon,9.0,9.0,9.0


> 만약 색인값을 DataFrame의 컬럼이나 Series의 색인에서 찾을 수 없다면 그 객체는 형식을 맞추기 위해 재색인된다.

In [85]:
series2 = pd.Series(range(3), index = list('bef'))

frame + series2

,b,d,e,f
Utah,0.0,NaN,3.0,NaN
Ohio,3.0,NaN,6.0,NaN
Texas,6.0,NaN,9.0,NaN
Oregon,9.0,NaN,12.0,NaN


> 만약 각 로우에 대해 연산을 수행하고 싶다면 산술 연산 메서드를 사용하면 된다.  
> 인자로 넘기는 axis 값은 `연산을 적용할 축 번호`다. axis = 'index'나 axis = 0은 DataFrame의 로우를 따라 연산을 수행하라는 의미다. 

In [86]:
series3 = frame['d']
series3

Utah       1.0
Ohio       4.0
Texas      7.0
Oregon    10.0
Name: d, dtype: float64

In [87]:
frame.sub(series3, axis = 'index')

,b,d,e
Utah,-1.0,0.0,1.0
Ohio,-1.0,0.0,1.0
Texas,-1.0,0.0,1.0
Oregon,-1.0,0.0,1.0


### 5.2.6 함수 적용과 매핑

- pandas 객체에도 NumPy의 유니버설 함수 (배열의 각 원소에 적용되는 메서드)를 적용할 수 있다.

In [88]:
frame = pd.DataFrame(np.random.randn(4,3),
                     columns = list('bde'), 
                     index = ['Utah','Ohio','Texas','Oregon'])
frame

,b,d,e
Utah,0.187983,0.465895,-0.701876
Ohio,0.659006,-1.197964,-1.157680
Texas,0.035218,-1.102666,-0.029607
Oregon,0.325372,0.345941,0.581992


In [89]:
np.abs(frame)

,b,d,e
Utah,0.187983,0.465895,0.701876
Ohio,0.659006,1.197964,1.157680
Texas,0.035218,1.102666,0.029607
Oregon,0.325372,0.345941,0.581992


> 자주 사용되는 또 다른 연산은 각 컬럼이나 로우의 1차원 배열에 함수를 적용하는 것.  
> DataFrame의 apply 메서드를 이용해 수행할 수 있다.

In [90]:
f = lambda x : x.max() - x.min()

frame.apply(f)

b    0.623788
d    1.663860
e    1.739672
dtype: float64

> apply 함수에 axis = 'columns' 인자를 넘기면 각 로우에 대해 한 번씩만 수행된다.

In [91]:
frame.apply(f, axis = 'columns')

Utah      1.167771
Ohio      1.856970
Texas     1.137884
Oregon    0.256620
dtype: float64

> apply 메서드에 전달된 함수는 스칼라값을 반환할 필요가 없다.  
> 여러 값을 가진 Series를 반환해도 된다.

In [92]:
def f(x) : 
    return pd.Series([x.min(), x.max()], index = ['min','max'])

frame.apply(f)

,b,d,e
min,0.035218,-1.197964,-1.157680
max,0.659006,0.465895,0.581992


> 배열의 각 원소에 적용되는 파이썬의 함수를 사용할 수도 있다.  
> frame 객체에서 실숫값을 문자열 포맷으로 변환하고 싶다면 applymap을 이용해서 다음과 같이 할 수 있다.  
> 메서드의 이름이 applymap인 이유는 Series는 각 원소에 적용할 함수를 지정하기 위한 map 메서드를 가지고 있기 때문이다.

In [93]:
format = lambda x: '%.2f' %x

frame.applymap(format)

,b,d,e
Utah,0.19,0.47,-0.70
Ohio,0.66,-1.20,-1.16
Texas,0.04,-1.10,-0.03
Oregon,0.33,0.35,0.58


In [94]:
frame['e'].map(format)

Utah      -0.70
Ohio      -1.16
Texas     -0.03
Oregon     0.58
Name: e, dtype: object

### 5.2.7 정렬과 순위

#### 정렬

> 로우나 컬럼의 색인을 알파벳순으로 정렬하려면 정렬된 `새로운 객체를 반환`하는 sort_index 메서드를 사용하면 된다.

In [95]:
obj = pd.Series(range(4), index = ['d','a','b','c'])
obj.sort_index()

a    1
b    2
c    3
d    0
dtype: int64

> DataFrame은 로우나 컬럼 중 하나의 축을 기준으로 정렬할 수 있다.

In [96]:
frame = pd.DataFrame(np.arange(8).reshape((2,4)), 
                     index = ['three','one'],
                     columns = ['d','a','b','c'])

frame.sort_index()

,d,a,b,c
one,4,5,6,7
three,0,1,2,3


In [97]:
frame.sort_index(axis = 1)

,a,b,c,d
three,1,2,3,0
one,5,6,7,4


> 데이터는 기본적으로 오름차순으로 정렬되고 내림차순으로 정렬할 수도 있다.

In [98]:
frame.sort_index(axis = 1, ascending = False)

,d,c,b,a
three,0,3,2,1
one,4,7,6,5


> Series 객체를 값에 따라 정렬하고 싶다면 sort_values 메서드를 사용하면 된다.  
> 정렬할 때 비어 잇는 값은 기본적으로 Series 객체에서 가장 마지막에 위치한다.

In [103]:
obj = pd.Series([4, np.nan, 7, np.nan, -3, 2])
obj.sort_values()

SyntaxError: unmatched ')' (<ipython-input-103-08b0cd034cbe>, line 2)

> DataFrame에서 하나 이상의 컬럼에 있는 `값`으로 정렬을 하는 경우 sort_values 함수의 by 옵션에 하나 이상의 컬럼을 넘기면 된다.

In [ ]:
frame = pd.DataFrame({'b' : [4,7,0-3,2], 
                      'a' : [0,1,0,1]})
frame.sort_values(by = ['a','b'])

#### 순위 

> `순위`는 1부터 배열의 유효한 데이터 개수까지 순서를 매긴다.  
> 기본적으로 Series와 DataFrame의 rank 메서드는 `동점인 항목`에 대해서는 `평균 순위`를 매긴다.

In [104]:
obj = pd.Series([7,-5,7,4,2,0,4])
obj.rank()    # 기본적으로 동점인 항목에 대하여는 평균 순위를 매긴다

0    6.5
1    1.0
2    6.5
3    4.5
4    3.0
5    2.0
6    4.5
dtype: float64

> 데이터 상에서 나타나는 순서에 따라 동률을 처리할 수도 있다.

In [105]:
obj.rank(method = "first")

0    6.0
1    1.0
2    7.0
3    4.0
4    3.0
5    2.0
6    5.0
dtype: float64

> DataFrame에서는 로우나 컬럼에 대해 순위를 정할 수 있다.

In [106]:
frame = pd.DataFrame({'b' : [4.3,7,-3,2], 
                      'a' : [0,1,0,1], 
                      'c' : [-2,5,8,-2.5]})
frame

,b,a,c
0,4.3,0,-2.0
1,7.0,1,5.0
2,-3.0,0,8.0
3,2.0,1,-2.5


In [107]:
frame.rank(axis = 'columns')

,b,a,c
0,3.0,2.0,1.0
1,3.0,1.0,2.0
2,1.0,2.0,3.0
3,3.0,2.0,1.0


##### 순위의 동률을 처리하는 메서드 

<details>
<summary>순위의 동률을 처리하는 메서드</summary>
<div markdown="1">

|메서드|설명|
|:--|:--|
|'average'|기본값. 같은 값을 가지는 항목들의 평균값을 순위로 삼는다.|
|'min'|같은 값을 가지는 그룹을 낮은 순위로 매긴다.|
|'max'|같은 값을 가지는 그룹을 높은 순위로 매긴다.|
|'first'|데이터 내의 위치에 따라 순위를 매긴다.|
|'dense'|method = 'min'과 같지만 같은 그룹 내에서 모두 같은 순위를 적용하지 않고 1씩 증가시킨다.|
    
</div>
</details>

### 5.2.8 중복 색인

- pandas의 많은 함수에서 색인값은 유일해야 하지만 의무적이지는 않다.

In [108]:
obj = pd.Series(range(5), index = list('aabbc'))
obj

a    0
a    1
b    2
b    3
c    4
dtype: int64

In [109]:
# is_unique 속성은 해당 값이 유일한지 아닌지 알려준다.
obj.index.is_unique

False

> 중복되는 색인값이 없을 때는 색인을 이용해서 데이터에 접급하면 스칼라 값을 반환한다.  
> 중복되는 색인값이 있을 때는 하나의 Series 객체를 반환한다.

In [110]:
obj['a']

a    0
a    1
dtype: int64

In [111]:
obj['c']

4

> DataFrame에서 로우를 선택하는 것도 동일하다.

In [112]:
df = pd.DataFrame(np.random.randn(4,3),
                  index = list('aabb'))
df

,0,1,2
a,-2.459429,1.630243,0.672751
a,-1.274175,1.544752,2.142934
b,-0.142194,-0.786047,-1.368971
b,0.403708,2.060002,-0.219961


In [113]:
df.loc['b']

,0,1,2
b,-0.142194,-0.786047,-1.368971
b,0.403708,2.060002,-0.219961


## 5.3 기술 통계 계산과 요약

- pandas 객체는 일반적인 수학 메서드와 통계 메서드를 가지고 있음.
- 순수 NumPy 배열에서 제공하는 동일한 메서드와 비교하여 pandas의 메서드는 처음부터 `누락된 데이터를 제외하도록` 설계되었다.

In [114]:
df = pd.DataFrame([[1.4,np.nan], 
                   [7.1,-4.5],
                   [np.nan,np.nan],
                   [0.75, -1.3]],
                  index = list('abcd'),
                  columns = ['one','two'])
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


> DataFrame의 sum 메서드를 호출하면 각 컬럼의 합을 담은 Series를 반환한다.

In [115]:
df.sum()

one    9.25
two   -5.80
dtype: float64

> axis = 'columns' 또는 axis = 1 옵션을 넘기면 각 컬럼의 합을 반환한다.

In [116]:
df.sum(axis = "columns")

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

> 전체 로우나 컬럼의 값이 NA가 아니라면 NA 값은 제외되고 계산된다.  
> 이는 skipna 옵션으로 조정할 수 있음

In [117]:
df.mean(axis = 'columns', skipna = False)

a      NaN
b    1.300
c      NaN
d   -0.275
dtype: float64

> idxmin이나 idxmax 같은 메서드는 최솟값 혹은 최댓값을 가지고 있는 색인값과 같은 간접 통계를 반환함.

In [118]:
df.idxmax()

one    b
two    d
dtype: object

> describe 메서드는 한 번에 여러 개의 통계 결과를 만들어 낸다.

In [119]:
df.describe()

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


> 수치 데이터가 아닐 경우 describe는 다른 요약 통계를 생성한다.

In [120]:
obj = pd.Series(['a','a','b','c']*4)
obj.describe()

count     16
unique     3
top        a
freq       8
dtype: object

#### 요약 통계 관련 메서드 

<details>
<summary>요약 통계 관련 메서드</summary>
<div markdown="1">

|메서드|설명|
|:--|:--|
|count|NA 값을 제외한 값의 수를 반환한다.|
|describe|Series나 DataFrame의 각 컬럼에 대한 요약 통계를 계산한다.|
|min, max|최솟값과 최댓값을 계산한다.|
|argmin, argmax|각각 최솟값과 최댓값을 담고 있는 색인의 위치(정수)를 반환한다.|
|idxmin, idxmax|각각 최솟값과 최댓값을 담고 있는 색인의 값을 반환한다.|
|quantile|0부터 1까지의 분위수를 계산한다.|
|sum|합을 계산한다.|
|mean|평균을 계산한다.|
|median|중간값을 반환한다.|
|mad|평균값에서 평균절대편차를 계산한다.|
|prod|모든 값의 곱|
|var|표본분산의 값을 계산한다.|
|std|표본표준편차의 값을 계산한다.|
|skew|표본비대칭도(3차 적률)의 값을 계산한다.|
|kurt|표본첨도(4차 적률)의 값을 계산한다.|
|cumsum|누적합을 계산한다.|
|cummin, cummax|각각 누적 최솟값과 누적 최댓값을 계산한다.|
|cumprod|누적곱을 계산한다.|
|diff|1차 산술차를 계산한다(시계열 데이터 처리 시 유용하다).|
|pct_change|퍼센트 변화율을 계산한다.|
    
</div>
</details>

### 5.3.1 상관관계와 공분산

- 상관관계나 공분산 같은 요약 통계 계산은 두 쌍의 인자를 필요로 한다.
- pandas-datareader 패키지를 이용해서 야후! 금융 사이트에서 구한 주식가겨과 시가총액을 담고 있는 다음 DataFrame을 생각해보자.

In [ ]:
# pip install pandas-datareader

In [122]:
import pandas_datareader.data as web

In [123]:
all_data = {ticker : web.get_data_yahoo(ticker)
            for ticker in ['AAPL','IBM','MSFT','GOOG']}
price = pd.DataFrame({ticker : data['Adj Close']
                      for ticker, data in all_data.items()})
volume = pd.DataFrame({ticker : data['Volume']
                      for ticker, data in all_data.items()})

In [124]:
price

,AAPL,IBM,MSFT,GOOG
Date,,,,
2016-08-16,25.645245,128.306335,53.246872,777.140015
2016-08-17,25.607737,128.098724,53.358116,779.909973
2016-08-18,25.574909,128.833298,53.395203,777.500000
2016-08-19,25.640558,127.779358,53.413731,775.419983
2016-08-22,25.441267,127.747444,53.460083,772.150024
...,...,...,...,...
2021-08-09,146.089996,141.250000,288.329987,2760.040039
2021-08-10,145.600006,141.380005,286.440002,2761.929932
2021-08-11,145.860001,142.130005,286.950012,2753.790039


> 각 주식의 퍼센트 변화율 계산하기

In [125]:
returns = price.pct_change()
returns.tail()

,AAPL,IBM,MSFT,GOOG
Date,,,,
2021-08-09,-0.000342,-0.008424,-0.003904,0.007049
2021-08-10,-0.003354,0.000920,-0.006555,0.000685
2021-08-11,0.001786,0.005305,0.001781,-0.002947
2021-08-12,0.020773,0.006614,0.009967,0.005084
2021-08-13,0.001410,0.000769,0.010490,0.000119


> corr : NA가 아니며 정렬된 색인에서 연속하는 두 Series에 대해 상관관계를 계산  
> cov : 공분산을 계산

In [126]:
returns['MSFT'].corr(returns['IBM'])

0.5174518665942109

In [127]:
returns['MSFT'].cov(returns['IBM'])

0.00014529595093979963

> 반면에 DataFrame에서 corr과 cov 메서드는 DataFrame 행렬에서 상관관계와 공분산을 계산한다.

In [128]:
returns.corr()

,AAPL,IBM,MSFT,GOOG
AAPL,1.000000,0.441103,0.735550,0.661975
IBM,0.441103,1.000000,0.517452,0.484141
MSFT,0.735550,0.517452,1.000000,0.775723
GOOG,0.661975,0.484141,0.775723,1.000000


In [129]:
returns.cov()

,AAPL,IBM,MSFT,GOOG
AAPL,0.000361,0.000137,0.000240,0.000212
IBM,0.000137,0.000268,0.000145,0.000133
MSFT,0.000240,0.000145,0.000294,0.000224
GOOG,0.000212,0.000133,0.000224,0.000283


> DataFrame의 corrwith 메서드를 사용하면 다른 Series나 DataFrame과의 상관관계를 계산한다.  
> Series를 넘기면 각 컬럼에 대해 계산한 상관관계를 담고 있는 Series를 반환한다.

In [130]:
returns.corrwith(returns['IBM'])

AAPL    0.441103
IBM     1.000000
MSFT    0.517452
GOOG    0.484141
dtype: float64

> DataFrame을 넘기면 맞아 떨어지는 컬럼 이름에 대한 상관관계를 계산한다.  
> 여기서는 시가총액의 퍼센트 변화율에 대한 거래량과의 상관관계를 계산함.

In [131]:
returns.corrwith(volume)

AAPL   -0.063093
IBM    -0.104074
MSFT   -0.057059
GOOG   -0.119371
dtype: float64

### 유일값, 값 세기, 멤버십

> unique 메서드는 중복되는 값을 제거하고 유일값만 담고 있는 Series를 반환한다.

In [132]:
obj = pd.Series(list('cadaabbcc'))
obj

0    c
1    a
2    d
3    a
4    a
5    b
6    b
7    c
8    c
dtype: object

In [133]:
uniques = obj.unique()
uniques

array(['c', 'a', 'd', 'b'], dtype=object)

> value_counts 메서드는 Series에서 $\text{도수}^{\text{frequency}}$를 계산하여 반환한다.

In [134]:
obj.value_counts()

a    3
c    3
b    2
d    1
dtype: int64

> value_counts에서 반환하는 Series는 담고 잇는 값을 내림차순으로 정렬한다.  
> value_counts메서드는 pandas의 `최상위 메서드`로, 어떤 배열이나 순차 자료구조에서도 사용할 수 있다.

In [135]:
pd.value_counts(obj.values, sort = False)

b    2
d    1
a    3
c    3
dtype: int64

> isin 메서드는 어떤 값이 Series에 존재하는지 나타내는 불리언 벡터를 반환함.  
> Series나 DataFrame의 컬럼에서 값을 골라내고 싶을 때 유용하게 사용함.

In [137]:
obj

0    c
1    a
2    d
3    a
4    a
5    b
6    b
7    c
8    c
dtype: object

In [138]:
mask = obj.isin(['b','c'])
mask

0     True
1    False
2    False
3    False
4    False
5     True
6     True
7     True
8     True
dtype: bool

In [139]:
obj[mask]

0    c
5    b
6    b
7    c
8    c
dtype: object

> DataFrame의 여러 컬럼에 대해 히스토그램을 구하는 경우

In [140]:
data = pd.DataFrame({'Qu1' : [1,3,4,3,4],
                     'Qu2' : [2,3,1,2,3],  
                     'Qu3' : [1,5,2,4,4]})
data

,Qu1,Qu2,Qu3
0,1,2,1
1,3,3,5
2,4,1,2
3,3,2,4
4,4,3,4


In [141]:
result = data.apply(pd.value_counts)
result

,Qu1,Qu2,Qu3
1,1.0,1.0,1.0
2,NaN,2.0,1.0
3,2.0,2.0,NaN
4,2.0,NaN,2.0
5,NaN,NaN,1.0
